In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation              
from scipy.fftpack import fft,ifft 

In [2]:
#define some functions we will be calling later on
def gaussian_wavepack(x, sigma, x0, k0):
    """Creates a Gaussian wavepack with momentum k0*hbar and width sigma."""
    return ((sigma * (np.pi)) ** (-0.5) * np.exp(-0.5 * ((x - x0) * 1. / sigma) ** 2 + 1j * x * k0))

def potential_barrier(x,width,heigth,center):
    """Creates a square potential barrier in domain x."""
    return heigth*(0.5*(np.sign((x-center)+width/2)+1) - 0.5*(np.sign((x-center)-width/2)+1))    

In [3]:
#define the class that we will be making instances of
class WaveFunction(object):
    def __init__(self, x, psi_x0, V,  hbar, m, t0, dt, k_range=0):
        """
        Args:
            x : array, float
                The domain of computations, a vector with x-coordinates.
            psi_x0 : array, complex
                The initial wavefunction, a vector with complex number corresponding to each position.
            V : array, float
                The potential, a vector with a float number corresponding to the potential at each coordinate
            hbar : float
                The value of the plank's constant, default 1
            m : float
                Mass of the particle, default 1
            t0 : float
                inital time, default 0
            dt : float
                The value of the timestep
            k_range : float
                This value specifies the range of the k values. For the default k_range = 0 a suitable range is
                used: [-0.5*N*dk,0.5*N*dk]
                Otherwise the range [-k_range, -k_range + N*dk]  is used. This might be useful if you know that wavenumbers
                outside of the default range will be non-zero.
        """
        #set internal params
        self.x = x
        self.V = V
        self.N = len(x)
        self.t = t0
        self.m = m
        self.hbar = hbar
        self.dx = self.x[1] - self.x[0]
        self.dk = 2 * np.pi / (self.N * self.dx)
        self.k =  -0.5 * self.N * self.dk + self.dk * np.arange(self.N)
        
        if k_range != 0:
            self.k = -k_range + self.dk * np.arange(self.N)
        
        #set initial psi_x and psi_k
        self.psi_x = psi_x0
        self.fft()
        
        #set evolution operators
        self.x_evolution_half = np.exp(-0.5 * 1j * self.V/ self.hbar * dt )
        self.k_evolution = np.exp(-0.5 * 1j * self.hbar/self.m * (self.k * self.k) * dt)
        
    #define functions for getters and setters and combine them in a property
    def _set_psi_x(self, psi_x):
        self.psi_fou_x = (psi_x * np.exp(-1j * self.k[0] * self.x)* self.dx / np.sqrt(2 * np.pi))
    def _get_psi_x(self):
        return (self.psi_fou_x * np.exp(1j * self.k[0] * self.x)* np.sqrt(2 * np.pi) / self.dx)

    def _set_psi_k(self, psi_k):
        self.psi_fou_k = psi_k * np.exp(1j * self.x[0]* self.dk * np.arange(self.N))
    def _get_psi_k(self):
        return self.psi_fou_k * np.exp(-1j * self.x[0] * self.dk * np.arange(self.N))
           
    psi_x = property(_get_psi_x, _set_psi_x)
    psi_k = property(_get_psi_k, _set_psi_k) 
    
    #define FFT and iFFT
    def fft(self):
        self.psi_fou_k = fft(self.psi_fou_x)
        
    def ifft(self):
        self.psi_fou_x = ifft(self.psi_fou_k)
    
    #calculate a timestep
    def time_step(self, dt, steps):
        for i in range(steps - 1):
            self.psi_fou_x *= self.x_evolution_half
            self.fft()
            self.psi_fou_k *= self.k_evolution
            self.ifft()
            self.psi_fou_x *= self.x_evolution_half

        self.fft()
        self.t += dt * steps

In [143]:
"""set-up simulation - uncomment/comment the relevant parts to toggle between different potential barrier setups. The setup
is now configured to observe resonant tunneling."""

#specify simple parameters
hbar = 1          #planck's constant /2pi
m = 0.5           #mass particle
t0 = 0            #start time
dt = 0.01         #length timestep
steps = 50        #number of timesteps between each frame
frames = 300      #not input param but needed for animation

#specify x-coordinates from [-xrange, xrange] with N number of points
N = 2000
xrange = 200
x = np.linspace(-xrange,xrange,N)

#specify potential
V0 = .15                            # Height of potential
a = 8                               # Width of potential
V = np.zeros(N)

#Single barrier
#V = potential_barrier(x, a, V0, 0)  # Potential barrier

#Single well
#V = -potential_barrier(x, a, V0, 0) # Potential well

#Double barrier distance dis apart
dis = 13.2
V = potential_barrier(x, a, V0, -dis/2) + potential_barrier(x, dis-a, -0.05*V0, 0) + potential_barrier(x, a, V0, dis/2)

#Add edges
#V[x < -98] = 1e10 
#V[x > 98] = 1e10

#specify initial wave
x0 = -50                            # Initial position
p0 = 0.7*np.sqrt(2*m*V0)            # Initial momentum (in terms of potential barrier energy)
d = hbar / np.sqrt(2*p0**2*(1/100)) # Width of wavefunction / 1/width of momenta
k0 = p0 / hbar
psi_x0 = gaussian_wavepack(x, d, x0, k0) #*(x-x0) #Uncomment for anti-symmetric wavefunction

#create instance of WaveFunction
W = WaveFunction(x=x, psi_x0=psi_x0, V=V, hbar=hbar, m=m, t0=0.0, dt=dt, k_range=0)

In [144]:
#set-up figure
fig = plt.figure()

xlim = [W.x[0],W.x[-1]]
ylim =[-0.1, 1.5*abs(W.psi_x).max()]
plot1 = fig.add_subplot(211, xlim=xlim, ylim=ylim)
plot1.set_xlabel('$x$')
plot1.set_ylabel('$|\psi(x)|$')
X, = plot1.plot([], [], c='r', label=r'$|\psi(x)|$')
V_graph, = plot1.plot([], [], c='k', label=r'$V(x)$')

klim = (W.k[0],W.k[-1])
ymin = abs(W.psi_k).min()
ymax = abs(W.psi_k).max()
plot2 = fig.add_subplot(212, xlim=klim, ylim=(ymin - 0.2 * (ymax - ymin), ymax + 0.2 * (ymax)))
plot2.set_xlabel('$k$')
plot2.set_ylabel('$| \psi(k) |$')
K, = plot2.plot([],[], c = 'r', label = r'$| \psi(k) |$')

In [145]:
#set-up animation
def initialize():
    X.set_data([],[])
    K.set_data([],[])
    V_graph.set_data([],[])
    return X, K, V_graph

def update(i):
    W.time_step(dt,steps)
    X.set_data(W.x, abs(W.psi_x))
    K.set_data(W.k, abs(W.psi_k))
    V_graph.set_data(W.x, W.V)
    return X, K, V_graph

#animate
ani = animation.FuncAnimation(fig, update,init_func=initialize,frames=frames, interval=30, blit=True)
plt.show()